In [1]:
import PyPDF2
from tqdm import tqdm
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(pdf_path)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

In [9]:
template = """Hello there. {}, {}"""
template.format(1,2)

'Hello there. 1, 2'

In [3]:
pdf_text = extract_text_from_pdf('../PDFs/harappan_history_ncert.pdf')

In [4]:
pdf_text = pdf_text.replace('\n',' ')

In [5]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    separators = ['. ','? ','! ',],
    chunk_size=3000,
    chunk_overlap=400,
    length_function=len,
    is_separator_regex=False,
)

In [7]:
texts = text_splitter.create_documents([pdf_text])

In [51]:
texts = [texts[i].page_content for i in range(len(texts))]

In [56]:
import os
import requests

def prompt_llm(prompt,model_name="meta-llama/Llama-2-70b-chat-hf",temperature=0.5,sess=None):
    url = "https://api.endpoints.anyscale.com/v1/chat/completions"
    api_key = "esecret_jd2ytygy3f7rzmj8pyr4yf2elx"
    if sess==None:
        sess = requests.Session()
    body = {
      "model": model_name,
      "messages": [{"role": "system", "content": "You're a helpful assistant"}, 
                   {"role": "user", "content": prompt}],
      "temperature": temperature
    }
    resp = sess.post(url, headers={"Authorization": f"Bearer {api_key}"}, json=body)
    return resp.json()['choices'][0]['message']['content'].strip()

In [57]:
context = texts[0]
prompt_template = f"""Context: {context}.\n Based on this context, create a short answered question 
                and answer for the question. First line should contain the question and next line answer."""
output = prompt_llm(prompt_template)

In [66]:
def get_prompt(content):
    prompt = f"""Content: {content}.\n Based on this content, create a short answered question and answer pair. Output should be in the template 'Q: <question> \n A: <answer>'"""
    return prompt

In [64]:
outputs = [output]
for content in tqdm(texts[1:]):
    prompt = get_prompt(content)
    outputs.append(prompt_llm(prompt))

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:39<00:00,  5.25s/it]


In [67]:
outputs2 = []
for context in tqdm(texts[:5]):
    prompt = get_prompt(context)
    outputs2.append(prompt_llm(prompt))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:19<00:00,  3.94s/it]


In [68]:
for output in outputs2:
    print(output)
    print('')

Q: What is the Harappan seal made of?
A: The Harappan seal is made of a stone called steatite.

Q: What types of grains were found at Harappan sites?
A: Wheat, barley, lentil, chickpea, sesame, and millets (in Gujarat) were found at Harappan sites. Rice was found in relatively low quantities.

Q: Did the Harappans use stone blades set in wooden handles or metal tools?
A: The Harappans used stone blades set in wooden handles, as well as metal tools.

Q: What is the evidence used by archaeologists to reconstruct dietary practices?
A: Archaeologists use artefacts such as querns, which were used for grinding grains and other foodstuffs, to reconstruct dietary practices.

Q: How were the tools used by the Harappans for harvesting?
A: The tools used by the Harappans for harvesting included stone and metal blades set in wooden handles, which were used for cutting and gathering crops.

Q: What is the significance of the reservoir found at Dholavira?
A: The reservoir found at Dholavira is signi

In [65]:
for output in outputs:
    print(output)
    print('')

Question: What is the Harappan seal made of?
Answer: The Harappan seal is made of a stone called steatite.

Q: What types of grains were found at Harappan sites?
A: Wheat, barley, lentil, chickpea, sesame, and millets (in Gujarat), as well as rice (though less common).

Question: Did the Harappans use stone blades set in wooden handles or did they use metal tools?

Answer: The Harappans used both stone blades set in wooden handles and metal tools. Archaeological evidence suggests that they used stone tools, such as saddle querns, for grinding and pounding, while metal tools, such as copper axes and sickles, were used for cutting and harvesting.

Q: What is the difference between the Citadel and the Lower Town in Mohenjodaro?
A: The Citadel was a smaller, higher section of the city that was walled and contained buildings constructed on mud brick platforms. The Lower Town was a larger, lower section of the city that was also walled and had buildings constructed on platforms, with a caref

In [14]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

In [13]:
letter

(612.0, 792.0)

In [25]:
text = """Q: What is the Harappan seal made of? 
A: The Harappan seal is made of a stone called steatite. 

Q: What types of grains were found at Harappan sites? 
A: Wheat, barley, lentil, chickpea, sesame, and millets (in Gujarat) were found at Harappan sites. Rice was found in relatively low quantities. 

Q: Did the Harappans use stone blades set in wooden handles or metal tools? 
A: The Harappans used stone blades set in wooden handles, as well as metal tools.

Q: What is the evidence used by archaeologists to reconstruct dietary practices?
A: Archaeologists use artefacts such as querns, which were used for grinding grains and other foodstuffs, to reconstruct dietary practices.

Q: How were the tools used by the Harappans for harvesting?
A: The tools used by the Harappans for harvesting included stone and metal blades set in wooden handles, which were used for cutting and gathering crops.

Q: What is the significance of the reservoir found at Dholavira?
A: The reservoir found at Dholavira is significant because it suggests that the Harappans had a sophisticated system of irrigation, which allowed them to farm in semi-arid regions."""

In [45]:
def convert_resp_text_to_qas(resp_text):
    resp_text_lines = resp_text.split('\n')
    question = ""
    answer = ""
    q = -1
    ans_lst = []
    q_lst = []
    for line in resp_text_lines:
        if line.strip()=="":
            continue
        if line.lower().strip().startswith('q:') or line.lower().strip().startswith('question'):
            if answer != "":
                ans_lst.append(answer.strip())
            question = line[2:]
            q = 1
        elif line.lower().strip().startswith('a:') or line.lower().strip().startswith('answer'):
            if question != "":
                q_lst.append(question.strip())
            q = 0
            answer = line[2:]
        elif q==1:
            question += " "
            question += line
        elif q==0:
            answer += " "
            answer += line
    return q_lst, ans_lst

In [47]:
q_lst, ans_lst = convert_resp_text_to_qas(text)

In [48]:
qa_list = [{"question": q, "answer": a} for q, a in zip(q_lst, ans_lst)]

In [49]:
c = canvas.Canvas("test.pdf",letter)
c.saveState()
y = 750
c.setFont("Helvetica-Bold",16)
c.drawString(60, y, 'Questions')
y -= 20
c.setFont("Helvetica",12)
for i in range(len(qa_list)):
    question = qa_list[i]['question']
    c.drawString(50, y, f"{i+1}. " + question)
    y -= 15
    if y <= 30:
        c.restoreState()
        c.saveState()
        y = 750

c.restoreState()
c.saveState()
y = 750
c.setFont("Helvetica-Bold",16)
c.drawString(60, y, 'Answers')
y -= 20
c.setFont("Helvetica",12)
for i in range(len(qa_list)):
    answer = qa_list[i]['answer']
    c.drawString(50, y, f"{i+1}. " + answer)
    y -= 15
    if y <= 30:
        c.restoreState()
        c.saveState()
        y = 750

c.save()


In [50]:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.rl_config import defaultPageSize
from reportlab.lib.units import inch
PAGE_HEIGHT=defaultPageSize[1]; PAGE_WIDTH=defaultPageSize[0]
styles = getSampleStyleSheet()


In [ ]:
Title = "Hello world"
pageinfo = "platypus example"
def myFirstPage(canvas, doc):
    canvas.saveState()
    canvas.setFont('Times-Bold',16)
    canvas.drawCentredString(PAGE_WIDTH/2.0, PAGE_HEIGHT-108, Title)
    canvas.setFont('Times-Roman',9)
    canvas.drawString(inch, 0.75 * inch, "First Page / %s" % pageinfo)
    canvas.restoreState()


